In [ ]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pysal as ps
import mplleaflet

In [ ]:
# This is our main data frame, containing both geometry and 
# attributes exported from CartoDB

data = gpd.read_file('neighborhood_nta_census.geojson')

In [ ]:
# This is only for the shapes that will be used by PySAL to 
# build the spatial weights matrix

psGeom = ps.open('neighborhood_nta_census/neighborhood_nta_census.shp', 'r')

In [ ]:
# We are building the spatial weight matrix and using the 
# neighborhood names as IDs of the matrix. Noted that we
# running a 'queen', shared vertices, neighborhood test.

W = ps.buildContiguity(psGeom, criterion='queen', ids=data['ntaname'].values.tolist())

In [ ]:
type(W)

In [ ]:
# Let's see how West Village is connected to other neighborhoods.
# Notice the weights.

W['West Village']

In [ ]:
# Now we would like to standardize all the weights. This can be 
# done by specifying 'R' as the matrix transformation. Then, let's
# look again the neighbors of the West Village. All the weights
# should add up to 1.

W.transform = 'R'
W['West Village']

In [ ]:
# Next, we're going to perform a spatial autocorrelation on the
# percent column. We first standardize the values by subtracting
# the mean and divide by the standard deviation.

Y = data['percent'].values
Y = (Y-Y.mean())/Y.std() # <<<---- normalization

In [ ]:
Y[:5]

In [ ]:
# and then compute the spatial lag for all neighborhoods based
# on the spatial weight matrix. We also store this as a column
# named 'w_percent' in the original table.

sl = ps.lag_spatial(W, Y)
data['w_percent'] = sl

In [ ]:
data.head()

In [ ]:
# Execute the Moran's I calculation

mi = ps.Moran(Y, W)

In [ ]:
# This is the Moran's I value, that would tell us whether population
# changes in New York are clustered, or not.

mi.I

In [ ]:
# Check the p-value of the calculation. This has to be < 0.05 for our
# calculation to be statistically significant.

mi.p_sim

In [ ]:
# It's time to look at the Moran Scatter Plot to inspet the results

f, ax = plt.subplots(1, figsize=(10,10))
sns.regplot(x='percent', y='w_percent', data=data)
plt.axvline(0, c='k', alpha=0.5)
plt.axhline(0, c='k', alpha=0.5)
plt.show()

In [ ]:
#### THE BELOW CODE SHOULD WORK FOR MOST INSTALLATION
f, ax = plt.subplots(1, figsize=(10,10))
data.plot(column='w_percent', scheme='QUANTILES', k=7, alpha=1, colormap='YlOrRd')
#mplleaflet.display(fig=f, crs=data.crs)

#### BUT IF NOT, PLEASE USE THE BELOW INSTEAD (and comment the previous blob)
# data.plot(column='w_percent', scheme='QUANTILES', k=7, alpha=1.0, colormap='YlOrRd', figsize=(10,10))
# mplleaflet.display(crs=data.crs)

In [ ]:
# Now, let's look at local indicator. Overall, we see some
# trends, but not so strong. Maybe a local indicator test
# could help us see in details how things are correlated.
# We run the Moran's LISA calculation provided by PySAL.

lisa = ps.Moran_Local(Y, W)

In [ ]:
# Let's narrow down to those neighborhoods that are
# statistically significant.

S = lisa.p_sim < 0.05

In [ ]:
# And which quadrants they belong to

Q = lisa.q

In [ ]:
# Next, we'll turn those into a GeoDataFrame for visualization.

records = map(lambda x: (data.iloc[x]['ntaname'], Q[x], data.geometry.iloc[x]),
              [i for i,s in enumerate(S) if s])


gdata = gpd.GeoDataFrame(records, columns=('ntaname', 'quadrant', 'geometry'))
gdata.head()

In [ ]:
# And plotting it with a basemap
f, ax = plt.subplots(1, figsize=(10,10))
gdata.plot(column='quadrant', scheme='QUANTILES', k=4, alpha=1.0, colormap='Blues')
#mplleaflet.display(fig=f, crs=gdata.crs)

##### Similar to the previous case, you can use the following if the above doesn't work
# gdata.plot(column='quadrant', scheme='QUANTILES', k=4, alpha=1.0, colormap='Blues', figsize=(10,10))
# mplleaflet.display(crs=gdata.crs)